# DWEN

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Auxiliary methods

## Configurações Globais

In [6]:
# %env epochs 1000
# %env base openoffice

env: epochs=1000
env: base=openoffice


In [7]:
MAX_SEQUENCE_LENGTH_T = 100 # 100
MAX_SEQUENCE_LENGTH_D = 100 # 500
'''
    Sequence length
    # Eclipse
    # 100   recall@25 = 
    # 20    recall@25 = 
    # Netbeans
    # 100   recall@25 = 
    # 20    recall@25 = 
    # Open office
    # 100   recall@25 : 0.16
    # 20    recall@25 : 0.20
'''

EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
freeze_train = .1 # 10% with freeze weights
best_loss = float('inf')
best_epoch = 0
verbose = 0
loss = 0

### Parse bugs preproprecessed

In [8]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'baseline_dwen_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Path embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [9]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [10]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [11]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [12]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

#### Loading bug ids in memory

In [13]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


361006

#### Dicionário de títulos e descrições

In [14]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 1min 20s, sys: 4.65 s, total: 1min 24s
Wall time: 1min 25s


#### Hashing bugs by buckets

In [15]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [16]:
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

In [17]:
baseline.train_data[:10]

[[275492, 218812],
 [288296, 264093],
 [273286, 293887],
 [57162, 62059],
 [82146, 67997],
 [56777, 61857],
 [169445, 165179],
 [250521, 273893],
 [247266, 241461],
 [36781, 38338]]

In [18]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

In [19]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '6\n',
 'bug_status': '2\n',
 'component': '361\n',
 'creation_ts': '2012-05-22 04:50:00 -0400',
 'delta_ts': '2012-07-09 06:49:56 -0400',
 'description': '[CLS] when a new scout project is created , the cs ##s theme of the platform is used by default . when the sw ##t client is started the background - image . / gt ##k ##gre ##y . p ##ng is not found . see also bug 37 ##9 ##9 ##8 ##9 . java . io . file ##not ##fo ##und ##ex ##ception : . / gt ##k ##gre ##y . p ##ng ( date ##i oder ve ##rz ##ei ##ch ##nis nic ##ht ge ##fu ##nden ) at java . io . file ##in ##put ##stream . open ( native method ) at java . io . file ##in ##put ##stream . < in ##it > ( file ##in ##put ##stream . java : 137 ) at org . eclipse . e ##4 . ui . cs ##s . core . ut ##il . imp ##l . resources . file ##res ##our ##ces ##lo ##cat ##ori ##mp ##l . get ##in ##put ##stream ( file ##res ##our ##ces ##lo ##cat ##ori ##mp ##l . java : 41 ) at org . eclipse . e ##4 . ui . cs ##s . core . ut ##il . imp ##l

### Generating the batch test

In [20]:
"Train ", len(baseline.dup_sets_train)

('Train ', 39339)

In [21]:
import random
random.choice(list(issues_by_buckets))

58671

In [22]:
110647 in experiment.baseline.bug_set

True

In [23]:
%%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = experiment.batch_iterator(None, 
                                                                                          baseline.train_data, 
                                                                                          baseline.dup_sets_train,
                                                                                          bug_train_ids,
                                                                                          batch_size_test, 1,
                                                                                          issues_by_buckets)

pos = np.full((1, batch_size_test), 1)
neg = np.full((1, batch_size_test), 0)
valid_sim = np.concatenate([pos, neg], -1)[0]

valid_title_sample_a = np.concatenate([valid_input_sample['title'], valid_input_sample['title']], 0)
valid_title_sample_b = np.concatenate([valid_input_pos['title'], valid_input_neg['title']], 0)
valid_desc_sample_a = np.concatenate([valid_input_sample['description'], valid_input_sample['description']], 0)
valid_desc_sample_b = np.concatenate([valid_input_pos['description'], valid_input_neg['description']], 0)

validation_sample = [valid_title_sample_a, valid_title_sample_b, valid_desc_sample_a, valid_desc_sample_b]

# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description'].shape[1]

CPU times: user 484 ms, sys: 0 ns, total: 484 ms
Wall time: 483 ms


In [24]:
len(valid_title_sample_a), len(valid_sim)

(256, 256)

In [25]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_sim.shape

((128, 100), (128, 100), (256,))

### Validar entrada

In [26]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

## Pre-trained embeddings

Loading pretrained word vectors

### Fasttext

In [27]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [28]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 21175'

In [29]:
def generating_embed(baseline, GLOVE_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')
    
    f2 = open(embed_path, 'rb')
    num_lines = sum(1 for line in f2)
    f2.close()
    
    f = open(embed_path, 'rb')
    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (num_lines + vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    i = 0
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embed = np.asarray(tokens[1:], dtype='float32')
        embeddings_index[word] = embed
        embedding_matrix[i] = embed
        loop.update(1)
        i+=1
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [30]:
%%time

generating_embed(baseline, GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 35s, sys: 3.33 s, total: 1min 39s
Wall time: 1min 38s


## Experiment

## Propose

https://github.com/tqtg/DuplicateBugFinder

In [31]:
from keras.initializers import RandomUniform, RandomNormal, Ones

### Embedding layer

In [32]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable, name):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer_{}'.format(name),
                                  weights=[embeddings],
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### DWEN model

In [33]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum, Subtract, \
    Average, GlobalAveragePooling1D, BatchNormalization, Activation
from keras.optimizers import Adam, Nadam
import keras.backend as K

def dwen_feature(title_feature_model, desc_feature_model, \
                  sequence_length_t, sequence_length_d, name):
    bug_t = Input(shape = (sequence_length_t, ), name = 'title_{}'.format(name))
    bug_d = Input(shape = (sequence_length_d, ), name = 'desc_{}'.format(name))
    
    # Embedding feature
    bug_t_feat = title_feature_model(bug_t)
    bug_d_feat = desc_feature_model(bug_d)
    
    bug_t_feat = GlobalAveragePooling1D()(bug_t_feat)
    bug_d_feat = GlobalAveragePooling1D()(bug_d_feat)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = Average(name = 'merge_features_{}'.format(name))([bug_t_feat, bug_d_feat])
    
    bug_feature_model = Model(inputs=[bug_t, bug_d], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

def dwen_model(bug_feature_output_a, bug_feature_output_b, name):
    
    inputs = np.concatenate([bug_feature_output_a.input, bug_feature_output_b.input], -1).tolist()
    
    bug_feature_output_a = bug_feature_output_a.output
    bug_feature_output_b = bug_feature_output_b.output
    
    # 2D concatenate feature
    bug_feature_output = concatenate([bug_feature_output_a, bug_feature_output_b])
    
    hidden_layers = 2
    
    # Deep Hidden MLPs
    for _ in range(hidden_layers):
        number_of_units = K.int_shape(bug_feature_output)[1]
        bug_feature_output = Dense(number_of_units // 2)(bug_feature_output)
#         bug_feature_output = BatchNormalization()(bug_feature_output)
        bug_feature_output = Activation('relu')(bug_feature_output)
        #bug_feature_output = Dropout(.5)(bug_feature_output)
    
     # Sigmoid
    output = Dense(1, activation='sigmoid')(bug_feature_output)

    similarity_model = Model(inputs=inputs, outputs=[output], name = 'dwen_output')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    optimizer = Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
    
    return similarity_model

In [34]:
import _pickle as pickle
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [35]:
# Domain to use
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
METHOD = 'baseline_dwen_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [36]:
%%time

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Embeddings
desc_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False, name='desc')
title_embedding_layer = embedding_layer(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.embedding_matrix), 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False, name='title')

# Similarity model
bug_feature_output_a = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_a')
bug_feature_output_b = dwen_feature(title_embedding_layer, desc_embedding_layer, 
                                    MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'dwen_b')
similarity_model = dwen_model(bug_feature_output_a, bug_feature_output_b, 'dwen')

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()


'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(None, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == limit_train): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, 
                                                        bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_test: {:.2f}".format(epoch+1, h[0], h_validation[0], h[1], h_validation[1]))
    
    loss = h[0]
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

#experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
#experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 100)          0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 100)          0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 100)          0                                            
__

Epoch: 8 Loss: 0.69, Loss_test: 0.71, acc: 0.51, acc_test: 0.50
Epoch: 9 Loss: 0.69, Loss_test: 0.70, acc: 0.53, acc_test: 0.51
=> result saved!
Epoch: 10 Loss: 0.68, Loss_test: 0.69, acc: 0.59, acc_test: 0.51
Epoch: 11 Loss: 0.68, Loss_test: 0.69, acc: 0.48, acc_test: 0.52
Epoch: 12 Loss: 0.69, Loss_test: 0.69, acc: 0.47, acc_test: 0.52
Epoch: 13 Loss: 0.69, Loss_test: 0.70, acc: 0.45, acc_test: 0.52
Epoch: 14 Loss: 0.68, Loss_test: 0.71, acc: 0.55, acc_test: 0.51
Epoch: 15 Loss: 0.67, Loss_test: 0.72, acc: 0.64, acc_test: 0.50
Epoch: 16 Loss: 0.67, Loss_test: 0.72, acc: 0.62, acc_test: 0.50
Epoch: 17 Loss: 0.68, Loss_test: 0.72, acc: 0.59, acc_test: 0.52
Epoch: 18 Loss: 0.68, Loss_test: 0.72, acc: 0.56, acc_test: 0.51
Epoch: 19 Loss: 0.67, Loss_test: 0.72, acc: 0.64, acc_test: 0.51
=> result saved!
Epoch: 20 Loss: 0.69, Loss_test: 0.72, acc: 0.55, acc_test: 0.51
Epoch: 21 Loss: 0.67, Loss_test: 0.73, acc: 0.59, acc_test: 0.51
Epoch: 22 Loss: 0.66, Loss_test: 0.74, acc: 0.63, acc_test

In [36]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_baseline_dwen_100.txt'

In [37]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [38]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

Saved model 'modelos/model_bert_preprocessing_baseline_dwen_100_feature_100epochs_64batch(openoffice).h5' to disk


In [39]:
len(result['train']), len(result['test'])

(100, 100)

In [40]:
model = similarity_model.get_layer('dense_3')
output = model.output
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
title_dwen_b (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_b (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_

In [41]:
# Domain to use
METHOD = 'baseline_dwen_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [42]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_input_pos, train_input_neg, \
            train_sim = experiment.batch_iterator(None, baseline.train_data, baseline.dup_sets_train, 
                                                  bug_train_ids, batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    
    num_batch = train_input_sample['title'].shape[0]
    pos = np.full((1, num_batch), 1)
    neg = np.full((1, num_batch), 0)
    train_sim = np.concatenate([pos, neg], -1)[0]
    
    title_sample_a = np.concatenate([train_input_sample['title'], train_input_sample['title']], 0)
    title_sample_b = np.concatenate([train_input_pos['title'], train_input_neg['title']], 0)
    desc_sample_a = np.concatenate([train_input_sample['description'], train_input_sample['description']], 0)
    desc_sample_b = np.concatenate([train_input_pos['description'], train_input_neg['description']], 0)
    train_batch = [title_sample_a, desc_sample_a, title_sample_b, desc_sample_b]
    
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == limit_train) ):
        save_loss(result)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, bug_train_ids, 'dwen')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_test: {:.2f}".format(epoch+1, h[0], h_validation[0], h[1], h_validation[1]))

Epoch: 101 Loss: 0.53, Loss_test: 0.70, acc: 0.73, acc_test: 0.57
Epoch: 102 Loss: 0.48, Loss_test: 0.70, acc: 0.75, acc_test: 0.56
Epoch: 103 Loss: 0.60, Loss_test: 0.73, acc: 0.70, acc_test: 0.60
Epoch: 104 Loss: 0.64, Loss_test: 0.76, acc: 0.65, acc_test: 0.61
Epoch: 105 Loss: 0.49, Loss_test: 0.70, acc: 0.71, acc_test: 0.57
Epoch: 106 Loss: 0.59, Loss_test: 0.70, acc: 0.69, acc_test: 0.59
Epoch: 107 Loss: 0.58, Loss_test: 0.70, acc: 0.66, acc_test: 0.59
Epoch: 108 Loss: 0.52, Loss_test: 0.69, acc: 0.76, acc_test: 0.59
Epoch: 109 Loss: 0.47, Loss_test: 0.68, acc: 0.80, acc_test: 0.57
=> result saved!
Epoch: 110 Loss: 0.55, Loss_test: 0.69, acc: 0.72, acc_test: 0.60
Epoch: 111 Loss: 0.47, Loss_test: 0.69, acc: 0.76, acc_test: 0.60
Epoch: 112 Loss: 0.49, Loss_test: 0.69, acc: 0.73, acc_test: 0.60
Epoch: 113 Loss: 0.48, Loss_test: 0.69, acc: 0.74, acc_test: 0.60
Epoch: 114 Loss: 0.49, Loss_test: 0.68, acc: 0.75, acc_test: 0.58
Epoch: 115 Loss: 0.59, Loss_test: 0.68, acc: 0.69, acc_test

Epoch: 223 Loss: 0.51, Loss_test: 0.70, acc: 0.76, acc_test: 0.60
Epoch: 224 Loss: 0.50, Loss_test: 0.69, acc: 0.75, acc_test: 0.59
Epoch: 225 Loss: 0.41, Loss_test: 0.70, acc: 0.81, acc_test: 0.60
Epoch: 226 Loss: 0.50, Loss_test: 0.70, acc: 0.73, acc_test: 0.60
Epoch: 227 Loss: 0.50, Loss_test: 0.70, acc: 0.75, acc_test: 0.59
Epoch: 228 Loss: 0.53, Loss_test: 0.70, acc: 0.70, acc_test: 0.59
Epoch: 229 Loss: 0.44, Loss_test: 0.71, acc: 0.77, acc_test: 0.59
=> result saved!
Epoch: 230 Loss: 0.51, Loss_test: 0.70, acc: 0.74, acc_test: 0.59
Epoch: 231 Loss: 0.52, Loss_test: 0.73, acc: 0.73, acc_test: 0.60
Epoch: 232 Loss: 0.46, Loss_test: 0.74, acc: 0.75, acc_test: 0.59
Epoch: 233 Loss: 0.49, Loss_test: 0.72, acc: 0.71, acc_test: 0.61
Epoch: 234 Loss: 0.51, Loss_test: 0.71, acc: 0.73, acc_test: 0.59
Epoch: 235 Loss: 0.52, Loss_test: 0.71, acc: 0.75, acc_test: 0.57
Epoch: 236 Loss: 0.50, Loss_test: 0.71, acc: 0.74, acc_test: 0.57
Epoch: 237 Loss: 0.53, Loss_test: 0.71, acc: 0.72, acc_test

Epoch: 345 Loss: 0.45, Loss_test: 0.70, acc: 0.77, acc_test: 0.57
Epoch: 346 Loss: 0.50, Loss_test: 0.70, acc: 0.76, acc_test: 0.57
Epoch: 347 Loss: 0.49, Loss_test: 0.70, acc: 0.74, acc_test: 0.59
Epoch: 348 Loss: 0.53, Loss_test: 0.70, acc: 0.73, acc_test: 0.59
Epoch: 349 Loss: 0.49, Loss_test: 0.69, acc: 0.71, acc_test: 0.59
=> result saved!
Epoch: 350 Loss: 0.51, Loss_test: 0.69, acc: 0.73, acc_test: 0.58
Epoch: 351 Loss: 0.41, Loss_test: 0.68, acc: 0.80, acc_test: 0.58
Epoch: 352 Loss: 0.41, Loss_test: 0.68, acc: 0.80, acc_test: 0.59
Epoch: 353 Loss: 0.40, Loss_test: 0.69, acc: 0.83, acc_test: 0.60
Epoch: 354 Loss: 0.44, Loss_test: 0.69, acc: 0.79, acc_test: 0.59
Epoch: 355 Loss: 0.43, Loss_test: 0.69, acc: 0.81, acc_test: 0.60
Epoch: 356 Loss: 0.47, Loss_test: 0.71, acc: 0.77, acc_test: 0.59
Epoch: 357 Loss: 0.53, Loss_test: 0.72, acc: 0.71, acc_test: 0.59
Epoch: 358 Loss: 0.47, Loss_test: 0.73, acc: 0.78, acc_test: 0.59
Epoch: 359 Loss: 0.52, Loss_test: 0.72, acc: 0.70, acc_test

Epoch: 467 Loss: 0.41, Loss_test: 0.68, acc: 0.81, acc_test: 0.61
Epoch: 468 Loss: 0.50, Loss_test: 0.69, acc: 0.77, acc_test: 0.61
Epoch: 469 Loss: 0.42, Loss_test: 0.68, acc: 0.77, acc_test: 0.60
=> result saved!
Epoch: 470 Loss: 0.47, Loss_test: 0.68, acc: 0.77, acc_test: 0.59
Epoch: 471 Loss: 0.40, Loss_test: 0.74, acc: 0.82, acc_test: 0.55
Epoch: 472 Loss: 0.56, Loss_test: 0.76, acc: 0.72, acc_test: 0.54
Epoch: 473 Loss: 0.41, Loss_test: 0.69, acc: 0.80, acc_test: 0.58
Epoch: 474 Loss: 0.49, Loss_test: 0.68, acc: 0.75, acc_test: 0.61
Epoch: 475 Loss: 0.52, Loss_test: 0.69, acc: 0.70, acc_test: 0.62
Epoch: 476 Loss: 0.51, Loss_test: 0.68, acc: 0.73, acc_test: 0.60
Epoch: 477 Loss: 0.43, Loss_test: 0.68, acc: 0.82, acc_test: 0.58
Epoch: 478 Loss: 0.45, Loss_test: 0.68, acc: 0.77, acc_test: 0.59
Epoch: 479 Loss: 0.51, Loss_test: 0.67, acc: 0.74, acc_test: 0.59
=> result saved!
Epoch: 480 Loss: 0.43, Loss_test: 0.69, acc: 0.78, acc_test: 0.64
Epoch: 481 Loss: 0.39, Loss_test: 0.70, ac

Epoch: 589 Loss: 0.41, Loss_test: 0.68, acc: 0.77, acc_test: 0.61
=> result saved!
Epoch: 590 Loss: 0.43, Loss_test: 0.69, acc: 0.83, acc_test: 0.60
Epoch: 591 Loss: 0.34, Loss_test: 0.70, acc: 0.84, acc_test: 0.61
Epoch: 592 Loss: 0.37, Loss_test: 0.67, acc: 0.84, acc_test: 0.62
Epoch: 593 Loss: 0.40, Loss_test: 0.66, acc: 0.84, acc_test: 0.63
Epoch: 594 Loss: 0.41, Loss_test: 0.66, acc: 0.78, acc_test: 0.63
Epoch: 595 Loss: 0.42, Loss_test: 0.67, acc: 0.81, acc_test: 0.64
Epoch: 596 Loss: 0.52, Loss_test: 0.67, acc: 0.73, acc_test: 0.62
Epoch: 597 Loss: 0.45, Loss_test: 0.69, acc: 0.78, acc_test: 0.60
Epoch: 598 Loss: 0.44, Loss_test: 0.69, acc: 0.80, acc_test: 0.60
Epoch: 599 Loss: 0.48, Loss_test: 0.68, acc: 0.77, acc_test: 0.61
=> result saved!
Epoch: 600 Loss: 0.47, Loss_test: 0.71, acc: 0.75, acc_test: 0.61
Epoch: 601 Loss: 0.45, Loss_test: 0.75, acc: 0.74, acc_test: 0.59
Epoch: 602 Loss: 0.44, Loss_test: 0.71, acc: 0.77, acc_test: 0.62
Epoch: 603 Loss: 0.41, Loss_test: 0.68, ac

Epoch: 711 Loss: 0.45, Loss_test: 0.63, acc: 0.77, acc_test: 0.65
Epoch: 712 Loss: 0.37, Loss_test: 0.64, acc: 0.81, acc_test: 0.62
Epoch: 713 Loss: 0.48, Loss_test: 0.64, acc: 0.77, acc_test: 0.62
Epoch: 714 Loss: 0.40, Loss_test: 0.63, acc: 0.79, acc_test: 0.62
Epoch: 715 Loss: 0.51, Loss_test: 0.64, acc: 0.72, acc_test: 0.61
Epoch: 716 Loss: 0.39, Loss_test: 0.66, acc: 0.80, acc_test: 0.61
Epoch: 717 Loss: 0.43, Loss_test: 0.65, acc: 0.77, acc_test: 0.62
Epoch: 718 Loss: 0.46, Loss_test: 0.65, acc: 0.80, acc_test: 0.61
Epoch: 719 Loss: 0.34, Loss_test: 0.68, acc: 0.84, acc_test: 0.61
=> result saved!
Epoch: 720 Loss: 0.35, Loss_test: 0.70, acc: 0.86, acc_test: 0.62
Epoch: 721 Loss: 0.44, Loss_test: 0.69, acc: 0.77, acc_test: 0.60
Epoch: 722 Loss: 0.43, Loss_test: 0.67, acc: 0.77, acc_test: 0.58
Epoch: 723 Loss: 0.40, Loss_test: 0.68, acc: 0.86, acc_test: 0.60
Epoch: 724 Loss: 0.42, Loss_test: 0.68, acc: 0.77, acc_test: 0.57
Epoch: 725 Loss: 0.44, Loss_test: 0.71, acc: 0.77, acc_test

Epoch: 833 Loss: 0.35, Loss_test: 0.67, acc: 0.83, acc_test: 0.55
Epoch: 834 Loss: 0.38, Loss_test: 0.68, acc: 0.83, acc_test: 0.55
Epoch: 835 Loss: 0.38, Loss_test: 0.69, acc: 0.84, acc_test: 0.55
Epoch: 836 Loss: 0.38, Loss_test: 0.69, acc: 0.83, acc_test: 0.55
Epoch: 837 Loss: 0.34, Loss_test: 0.69, acc: 0.83, acc_test: 0.58
Epoch: 838 Loss: 0.35, Loss_test: 0.70, acc: 0.84, acc_test: 0.59
Epoch: 839 Loss: 0.33, Loss_test: 0.70, acc: 0.86, acc_test: 0.59
=> result saved!
Epoch: 840 Loss: 0.32, Loss_test: 0.71, acc: 0.84, acc_test: 0.59
Epoch: 841 Loss: 0.48, Loss_test: 0.71, acc: 0.72, acc_test: 0.60
Epoch: 842 Loss: 0.40, Loss_test: 0.70, acc: 0.82, acc_test: 0.58
Epoch: 843 Loss: 0.32, Loss_test: 0.69, acc: 0.84, acc_test: 0.58
Epoch: 844 Loss: 0.45, Loss_test: 0.68, acc: 0.79, acc_test: 0.58
Epoch: 845 Loss: 0.37, Loss_test: 0.67, acc: 0.84, acc_test: 0.58
Epoch: 846 Loss: 0.38, Loss_test: 0.66, acc: 0.80, acc_test: 0.58
Epoch: 847 Loss: 0.33, Loss_test: 0.66, acc: 0.84, acc_test

In [43]:
len(result['train']), len(result['test'])

(900, 900)

In [44]:
encoded = model.get_layer('merge_features_dwen_a')
output = encoded.output
inputs = similarity_model.inputs[:-2]
bug_feature_output_a = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [45]:
inputs

[<tf.Tensor 'title_dwen_a:0' shape=(?, 20) dtype=float32>,
 <tf.Tensor 'desc_dwen_a:0' shape=(?, 20) dtype=float32>]

In [46]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_baseline_dwen_1000_feature1000epochs_64batch(openoffice)'

In [47]:
experiment.save_model(model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
experiment.save_model(bug_feature_output_a, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
"Model saved"

Saved model 'modelos/model_bert_preprocessing_baseline_dwen_1000_feature_1000epochs_64batch(openoffice).h5' to disk


'Model saved'

In [48]:
recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, bug_feature_output_a, issues_by_buckets, bug_train_ids, 'dwen')
print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, acc: {:.2f}, acc_tets: {:.2f}, recall@25: {:.2f}".format(epoch+1, h[0], h_validation[0],  h[1], h_validation[1], recall))

Epoch: 900 Loss: 0.49, Loss_test: 0.72, acc: 0.74, acc_tets: 0.56, recall@25: 0.20


In [49]:
exported_rank[:20]

['98306:88871,50853,33630,90791|22485:0.8136442601680756,21975:0.8088147938251495,52149:0.8051595985889435,21976:0.802871823310852,12936:0.7992842644453049,55057:0.7969578802585602,20737:0.795499861240387,44398:0.7953686863183975,33974:0.7950585186481476,47054:0.7949663251638412,5788:0.7920135408639908,9850:0.7912836819887161,52219:0.7911051213741302,16049:0.7901453077793121,16050:0.7901453077793121,39098:0.7892025262117386,98670:0.7888880670070648,51118:0.7872904539108276,14063:0.7869907915592194,57446:0.7842895835638046,26997:0.7840646505355835,45068:0.783794566988945,7724:0.7837286591529846,11265:0.7834534645080566,86695:0.7829296588897705,61548:0.7827413380146027,1569:0.7823092937469482,45406:0.7822113484144211,10469:0.7822065055370331',
 '32771:32490,33548,32560,33879,32699|89982:0.8311087787151337,85708:0.8213895708322525,104067:0.8208613246679306,5745:0.8162567317485809,100550:0.8141317218542099,52506:0.8139511346817017,112299:0.8136376440525055,22038:0.8124389946460724,10306:0.

### Retrieval evaluation

In [50]:
print("Total of queries:", len(retrieval.test))

Total of queries: 8265


#### Getting the model trained

In [51]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_baseline_dwen_1000_feature_1000epochs_64batch(openoffice)'

In [52]:
bug_feature_output_a.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
title_dwen_a (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
desc_dwen_a (InputLayer)        (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_layer_title (Embeddin (None, 20, 300)      581257500   title_dwen_a[0][0]               
__________________________________________________________________________________________________
embedding_layer_desc (Embedding (None, 20, 300)      581257500   desc_dwen_a[0][0]                
__________________________________________________________________________________________________
global_ave

In [53]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_baseline_dwen_1000.txt'

In [54]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [55]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.15,
 '2 - recall_at_10': 0.17,
 '3 - recall_at_15': 0.18,
 '4 - recall_at_20': 0.19,
 '5 - recall_at_25': 0.2}